In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg16

In [ ]:
def create_vgg_base():
    base_model = vgg16(pretrained=True)
    base_layers = list(base_model.features)[:30]  # Up to conv5_3 layer

    # Modify the architecture
    base_layers[16] = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)  # FC6 with atrous
    base_layers[23] = nn.Conv2d(1024, 1024, kernel_size=1)  # FC7 as conv

    # Replace FC6 and FC7 in VGG16 with convolutions
    base_layers.append(nn.MaxPool2d(kernel_size=3, stride=1, padding=1))  # Add modified pool5
    base_layers = nn.ModuleList(base_layers)
    
    return base_layers

In [ ]:
def add_extras():
    # Configuration for extra layers
    # (256, 'S') means 256 channels and stride 2. 'S' indicates a stride of 2.
    extras_cfg = [256, 'S', 512, 128, 'S', 256, 128, 256, 128, 256]
    layers = []
    in_channels = 1024  # Output channels from the last layer of VGG base
    
    for k, v in enumerate(extras_cfg):
        if v == 'S':
            layers += [nn.Conv2d(in_channels, extras_cfg[k+1], kernel_size=(1, 3)[k % 2], stride=2, padding=1)]
        else:
            layers += [nn.Conv2d(in_channels, v, kernel_size=(1, 3)[k % 2])]
        in_channels = v

    return nn.ModuleList(layers)

In [ ]:
def multibox(vgg_base, extra_layers, num_classes):
    loc_layers = []
    conf_layers = []
    vgg_source = [21, -2]  # conv4_3 and conv7 (FC7) layer indices in the base

    for k, v in enumerate(vgg_source):
        loc_layers += [nn.Conv2d(vgg_base[v].out_channels, 4 * 4, kernel_size=3, padding=1)]
        conf_layers += [nn.Conv2d(vgg_base[v].out_channels, 4 * num_classes, kernel_size=3, padding=1)]

    for k, v in enumerate(extra_layers[1::2], 2):  # From conv8_2 onwards, at every alternate layer
        loc_layers += [nn.Conv2d(v.out_channels, 4 * 4, kernel_size=3, padding=1)]
        conf_layers += [nn.Conv2d(v.out_channels, 4 * num_classes, kernel_size=3, padding=1)]

    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)

In [ ]:
class SSD(nn.Module):
    def __init__(self, num_classes):
        super(SSD, self).__init__()
        self.num_classes = num_classes
        self.vgg_base = create_vgg_base()
        self.extras = add_extras()
        self.loc, self.conf = multibox(self.vgg_base, self.extras, num_classes)

        self.init_weights()

    def forward(self, x):
        # Define the forward pass
        pass

    def init_weights(self):
        # Initialize weights
        pass